In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-interweave/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

725 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/musique-mini',
   'name': 'answerable',
   'split': 'validation'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.5,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'auto-interweave/direct.json',
   'n_shot': 0},
  'run': 1},
 'metrics': {'exact_match': 0.5333333333333333,
  'f1': 0.6531262626262626,
  'fuzzy_match': 0.6033333333333334,
  '2hops': {'exact_match': 0.61, 'f1': 0.724955266955267, 'fuzzy_match': 0.68},
  '3hops': {'exact_match': 0.57,
   'f1': 0.6818571428571428,
   'fuzzy_match': 0.67},
  '4hops': {'exact_match': 0.42,
   'f1': 0.5525663780663781,
   'fuzzy_match': 0.46},
  'gen_token_count': {'all': {'count': 300.0,
    'mean': 5.986666666666666,
    'std': 2.0867047414080746,
    'min': 3.0,
    '25%': 5.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 17.0},
   'success': {'count': 192.0,
    'mean': 5.895

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep") 
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

499 experiments after preprocessing


In [4]:
def parse_qa_technique(system_prompt_filename):
    return system_prompt_filename.split("/")[0]

df["params.qa.technique"] = df["params.qa.system_prompt"].map(parse_qa_technique)

In [5]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.temperature,params.run
0,ccot/excellent-format-thought.txt,cq.txt,auto-interweave/ccot.json,0,0.1,"(2,)"
1,ccot/excellent-format-thought.txt,cq.txt,auto-interweave/ccot.json,0,0.7,"(2,)"
2,ccot/excellent-format-thought.txt,cq.txt,auto-interweave/ccot.json,0,1.5,"(1,)"
3,ccot/excellent-format-thought.txt,cq.txt,auto-interweave/ccot.json,0,2.0,"(3, 3)"
4,ccot/excellent-format-thought.txt,cq.txt,auto-interweave/ccot.json,1,0.1,"(1, 3)"
...,...,...,...,...,...,...
172,direct/excellent-format-few.txt,cq.txt,auto-interweave/direct.json,0,0.1,"(1, 2, 3, 3)"
188,direct/excellent-format-few.txt,cq.txt,auto-interweave/direct.json,2,2.0,"(2,)"
194,direct/excellent-format-few.txt,cq.txt,auto-interweave/direct.json,3,2.0,"(2,)"
200,direct/excellent-format-few.txt,cq.txt,auto-interweave/direct.json,6,2.0,"(2,)"


In [6]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/excellent-format-thought.txt
	ccot/format-thought.txt
	cot/excellent-format-thought.txt
	cot/format-thought.txt
	cte/excellent-format-triplets-few.txt
	cte/format-triplets-few.txt
	direct/excellent-format-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	auto-interweave/ccot.json
	auto-interweave/cot.json
	auto-interweave/cte.json
	auto-interweave/direct.json
- params.qa.n_shot
	0
	1
	2
	3
	6
	9
- params.qa.temperature
	0.1
	0.7
	1.5
	2.0
	0.5
	1.0
- params.run
	(2,)
	(1,)
	(3, 3)
	(1, 3)
	(3,)
	(1, 1, 3)
	(2, 2)
	(1, 2, 3)
	(2, 3)
	(1, 2)
	(1, 2, 3, 3)


In [7]:
df.to_json('results.jsonl', orient='records', lines=True)